# Demo for using the code base

## We load the different packages

In [1]:
import pandas as pd
import datetime as dt
import numpy
import py_vollib_vectorized

from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta


In [2]:
# importing functions from the code base
base_path = os.path.abspath(os.path.join('..'))
print(base_path)
if base_path not in sys.path:
    sys.path.append(base_path)

import src.api as api


/Users/marcfiani/Documents/projects/active/crypto_option_dashboard


## Getting the instruments

The following function allow us to get the instruments available in Deribit

In [9]:
import nest_asyncio
nest_asyncio.apply()
instruments = api.get_instruments("BTC", kind="option", expired=False)
instruments.head()

,tick_size,taker_commission,strike,settlement_period,rfq,quote_currency,option_type,min_trade_amount,maker_commission,kind,is_active,instrument_name,expiration_timestamp,creation_timestamp,contract_size,block_trade_commission,base_currency,creation_time,expiration_time
0,0.0005,0.0003,55000.0,month,False,BTC,call,0.1,0.0003,option,True,BTC-24JUN22-55000-C,1656057600000,1629483660000,1.0,0.00015,BTC,2021-08-20 13:21:00,2022-06-24 03:00:00
1,0.0005,0.0003,33000.0,day,False,BTC,call,0.1,0.0003,option,True,BTC-25JAN22-33000-C,1643097600000,1642924860000,1.0,0.00015,BTC,2022-01-23 03:01:00,2022-01-25 03:00:00
2,0.0005,0.0003,30000.0,month,False,BTC,call,0.1,0.0003,option,True,BTC-24JUN22-30000-C,1656057600000,1627994072000,1.0,0.00015,BTC,2021-08-03 07:34:32,2022-06-24 03:00:00
3,0.0005,0.0003,55000.0,month,False,BTC,call,0.1,0.0003,option,True,BTC-28JAN22-55000-C,1643356800000,1635292800000,1.0,0.00015,BTC,2021-10-26 19:00:00,2022-01-28 03:00:00
4,0.0005,0.0003,45000.0,month,False,BTC,put,0.1,0.0003,option,True,BTC-25MAR22-45000-P,1648195200000,1628197380000,1.0,0.00015,BTC,2021-08-05 16:03:00,2022-03-25 03:00:00


In [4]:
option=pd.read_csv("../files/BTC-15OCT21-50000-P.csv")
spot=pd.read_csv("../files/BTC-PERPETUAL.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../files/BTC-15OCT21-50000-P.csv'

In [ ]:
option.head()

,Unnamed: 0,volume,ticks,status,open,low,high,cost,close,timestamp
0,0,3.0,1633046400000,ok,0.159,0.1590,0.159,0.4770,0.1590,2021-10-01 02:00:00
1,1,0.0,1633132800000,ok,0.159,0.1590,0.159,0.0000,0.1590,2021-10-02 02:00:00
2,2,0.0,1633219200000,ok,0.159,0.1590,0.159,0.0000,0.1590,2021-10-03 02:00:00
3,3,6.7,1633305600000,ok,0.094,0.0625,0.094,0.4219,0.0625,2021-10-04 02:00:00
4,4,37.2,1633392000000,ok,0.057,0.0345,0.057,1.7432,0.0345,2021-10-05 02:00:00


In [ ]:
data=pd.merge(option,spot,how="left",on="ticks",suffixes=("_option","_spot"))

In [ ]:
unwanted = data.columns[data.columns.str.startswith('Unnamed')]
data.drop(unwanted, axis=1, inplace=True)
data.drop(["timestamp_option","timestamp_spot"],axis=1)
data['timestamp']=data.ticks.apply(lambda x:dt.datetime.fromtimestamp(x/1000))


In [ ]:
strike=50000 #from instrument name
option_type="p" # from instrument name or can be got from get_intrument function
expiry=dt.datetime(2021,10,15)
r=0.01 # ask nico about this
qty = 1277
mult=1
is_delta_hedged=True

In [ ]:
data["time_to_expiry"]=data.timestamp.apply(lambda x:expiry-x)
data["days_to_expiry"]=data["time_to_expiry"].apply(lambda x: x.days)

In [ ]:
data['ivol_mid']=implied_volatility(data['close_option'],data['close_spot'],
    strike,data['days_to_expiry']/365,r,option_type,return_as="series")

/Users/marcfiani/Documents/projects/active/python_backtester_nico/venv/lib/python3.9/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning: Found Below Intrinsic contracts at index [0, 1, 2, 3]
  below_intrinsic, above_max_price = _check_below_and_above_intrinsic(K, F, flag, undiscounted_option_price, on_error)


In [ ]:
data['delta_mid'] = py_vollib_vectorized.greeks.delta(option_type, data['close_spot'], 
        strike,data['days_to_expiry']/365, r, data[ 'ivol_mid'])


In [ ]:
data['daily_pnl_option']=data['close_option'].diff()*qty*mult
data["daily_pnl_spot"] = data['close_spot'].diff() * data['delta_mid'].shift()*qty*mult
ts=data['daily_pnl_option']- data["daily_pnl_spot"]*is_delta_hedged
data['pnl']=ts.cumsum()